In [1]:
%pwd

'e:\\Test MLOPS\\Project_Classification\\chicken-Disease\\research'

In [2]:
import os

os.chdir('../')

%pwd

'e:\\Test MLOPS\\Project_Classification\\chicken-Disease'

In [5]:
#entity
from dataclasses import dataclass
from pathlib import Path

@dataclass
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path 

    

In [6]:
#configuration manager
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(self,config_file_path = CONFIG_FILE_PATH,params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path) #returns configbox objects
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [7]:
#create component
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
import tensorflow as tf
import time

[2023-06-12 10:37:00,690: INFO: utils: NumExpr defaulting to 8 threads.]


In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-06-12 11:00:35,880: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-12 11:00:36,005: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-12 11:00:36,012: INFO: common: created directory at: artifacts]
[2023-06-12 11:00:36,016: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-06-12 11:00:36,017: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]


In [ ]:
#no pipeline is required as it will be directly called during the training process